<a href="https://colab.research.google.com/github/roghayefazli/clustering-Text-Mining/blob/main/text_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

https://zenodo.org/records/5853572

In [ ]:
data = pd.read_excel('Question.xlsx')
data.head()

BadZipFile: File is not a zip file

In [ ]:
data.info()

make dataframe

In [ ]:
x = data[['Body']]
x

delete nan

In [ ]:
x=x.dropna()
x

count most repetitive words

In [ ]:
from collections import Counter

In [ ]:
Counter(' '.join(x['Body']).split()).most_common(30)

make stopwords file

In [ ]:
with open('stopwords.txt') as stopwords_file:
    stopwords = stopwords_file.readlines()
stopwords = [line.replace('\n', '') for line in stopwords]
stopwords

delete stopwords

In [ ]:
from nltk import word_tokenize
import nltk
import re
nltk.download('punkt')
dataset = pd.DataFrame(columns=['title_body'])
for index, row in x.iterrows():
    title_body_tokenized = word_tokenize(row['Body'])
    title_body_tokenized_filtered = [w.lower() for w in title_body_tokenized if not w.lower() in stopwords]
    #     title_body_tokenized_filtered_stemmed = [stemmer.stem(w) for w in title_body_tokenized_filtered]
    s = re.sub('[^\w\s]', '', ' '.join(title_body_tokenized_filtered))
    s = re.sub("\d+", "", s)
    dataset.loc[index] = {
        'title_body': s,
    }

In [ ]:
dataset

using text TfIdf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer()
vectorizer.fit(dataset['title_body'])

In [ ]:
x = vectorizer.transform(dataset['title_body'])

In [ ]:
x

kmeans algorithm

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=10 , max_iter=300)
kmeans.fit(x)

In [ ]:
kmeans.inertia_

check scores

In [ ]:
from sklearn.metrics import silhouette_score,davies_bouldin_score
labels = kmeans.labels_
s = silhouette_score(x,labels)
print(f'silhouette score : {s} ')

make dataFrame

In [ ]:
xl = dataset[['title_body']]

make labels column

In [ ]:
xl['cluster'] = kmeans.labels_

In [ ]:
xl[xl['cluster']==7]

In [ ]:
import seaborn as sns

In [ ]:
sns.countplot(data=xl, x='cluster')

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(x = labels , y = xl)

In [ ]:
pip install umap-learning

In [ ]:
import umap
reducer = umap.UMAP(n_components=1000)

In [ ]:
X_reduced = reducer.fit_transform(x)

In [ ]:
import numpy as np
np.shape(X_reduced)

meanshift algorithm.

In [ ]:
from sklearn.cluster import MeanShift

In [ ]:
msh = MeanShift(bandwidth = 0.85)
msh.fit(X_reduced)

In [ ]:
labels = msh.labels_
labels

In [ ]:
xl['cluster'] = msh.labels_

In [ ]:
sns.countplot(data=xl, x='cluster')

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
dbscan = DBSCAN(eps=0.3,min_samples = 20)
dbscan.fit(X_reduced)
label = dbscan.labels_
label = list(label)
print(f'noise : {label.count(-1)}')

In [ ]:
xl['cluster'] = label
xl

if label = -1 : we have noise

In [ ]:
dbscan = DBSCAN(eps=0.6,min_samples = 20)
dbscan.fit(X_reduced)
label = dbscan.labels_
label = list(label)
print(f'noise : label.count(-1)')

In [ ]:
xl['cluster'] = label
xl

In [ ]:
sns.countplot(data=xl, x='cluster')